<a href="https://colab.research.google.com/github/TasnubaTiha33/Wav2vec2/blob/Updated-wav2vec2/Wav2vec2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install transformers datasets evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
from huggingface_hub import notebook_login

notebook_login()

**Loading Base wav2vec2 model**

In [ ]:
from transformers import AutoFeatureExtractor

feature_extractor = AutoFeatureExtractor.from_pretrained("facebook/wav2vec2-base")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/159 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.84k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/configuration_utils.py:364: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


In [ ]:
%%capture
!pip install gdown

In [ ]:
import os
import zipfile
from google.colab import files
import time
import json
import librosa
from datasets import Dataset, DatasetDict, Audio
import glob
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [ ]:
%%capture
!gdown --id 1EZTvcHo2x3cim4uSTkfMHoagipfQtLak
time.sleep(5)

!unzip /content/FESC_CaFE.zip

In [ ]:
import json

# Paths to train and test JSON files
# train_json_path = os.path.join(extract_path, 'train.json')
# test_json_path = os.path.join(extract_path, 'test.json')

# Load JSON files
with open("/content/CaFE/CaFE_splits/train.json",'r') as f:
    train_data = json.load(f)

with open("/content/CaFE/CaFE_splits/test.json",'r') as f:
    test_data = json.load(f)

In [ ]:
# train_data

In [ ]:
train_samples_list = [
    {"id": sample_id, **sample_data} for sample_id, sample_data in train_data.items()
]
train_dataset = Dataset.from_list(train_samples_list)

In [ ]:
test_samples_list = [
    {"id": sample_id, **sample_data} for sample_id, sample_data in test_data.items()
]
test_dataset = Dataset.from_list(test_samples_list)

In [ ]:
dataset = DatasetDict({
    'train' : train_dataset,
    'test' : test_dataset,
    'dev' : test_dataset
})
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'wav', 'length', 'emo'],
        num_rows: 780
    })
    test: Dataset({
        features: ['id', 'wav', 'length', 'emo'],
        num_rows: 156
    })
    dev: Dataset({
        features: ['id', 'wav', 'length', 'emo'],
        num_rows: 156
    })
})

In [ ]:
def add_base_path(example):
    example['wav'] = '/content/CaFE/' + example['wav']  # Adjust column name if it's not 'wav'
    return example

dataset = dataset.map(add_base_path, batched=False)
dataset['train'][0]

Map:   0%|          | 0/780 [00:00<?, ? examples/s]

Map:   0%|          | 0/156 [00:00<?, ? examples/s]

Map:   0%|          | 0/156 [00:00<?, ? examples/s]

{'id': '09-J-1-4',
 'wav': '/content/CaFE/CaFE_48k/Happiness/Faible/09-J-1-4.wav',
 'length': 3.5731875,
 'emo': 'happiness'}

In [ ]:
import librosa
def load_audio(example):
  audio_array, sample_rate = librosa.load(example['wav'], sr=16000)
  example["audio_array"] = audio_array
  example["sample_rate"] = sample_rate
  return example

In [ ]:
dataset = dataset.map(load_audio, batched=False)
dataset

Map:   0%|          | 0/780 [00:00<?, ? examples/s]

Map:   0%|          | 0/156 [00:00<?, ? examples/s]

Map:   0%|          | 0/156 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'wav', 'length', 'emo', 'audio_array', 'sample_rate'],
        num_rows: 780
    })
    test: Dataset({
        features: ['id', 'wav', 'length', 'emo', 'audio_array', 'sample_rate'],
        num_rows: 156
    })
    dev: Dataset({
        features: ['id', 'wav', 'length', 'emo', 'audio_array', 'sample_rate'],
        num_rows: 156
    })
})

In [ ]:
dataset['train'][0].keys()

dict_keys(['id', 'wav', 'length', 'emo', 'audio_array', 'sample_rate'])

Import **Libraries**

In [ ]:
import torch
from transformers import Wav2Vec2ForSequenceClassification, Wav2Vec2Processor, TrainingArguments, Trainer
!pip install datasets --upgrade

In [ ]:
# Define labels and mappings for your dataset
labels = ['anger', 'happiness', 'neutral', 'sadness']
label2id = {label: i for i, label in enumerate(labels)}
id2label = {i: label for i, label in enumerate(labels)}

In [ ]:
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")
# Filter out examples with labels that are not in label2id
dataset = dataset.filter(lambda x: x['emo'] in label2id.keys())
# Preprocess function to convert audio to model-compatible inputs
def preprocess_function(batch):
    # Process audio array into input values for the model
    batch["input_values"] = processor(batch["audio_array"], sampling_rate=16000).input_values[0]
    # Convert label to integer id
    batch["labels"] = label2id[batch["emo"]]
    return batch

# Map preprocess function to the dataset
dataset = dataset.map(preprocess_function, remove_columns=["audio_array", "sample_rate", "wav"])


preprocessor_config.json:   0%|          | 0.00/159 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/163 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.60k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/291 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

...

Filter:   0%|          | 0/156 [00:00<?, ? examples/s]

Filter:   0%|          | 0/156 [00:00<?, ? examples/s]

Map:   0%|          | 0/420 [00:00<?, ? examples/s]

Map:   0%|          | 0/84 [00:00<?, ? examples/s]

Map:   0%|          | 0/84 [00:00<?, ? examples/s]

In [ ]:
import evaluate
accuracy = evaluate.load("accuracy")

In [ ]:
import numpy as np


def compute_metrics(eval_pred):
    predictions = np.argmax(eval_pred.predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=eval_pred.label_ids)

In [ ]:
from transformers import AutoModelForAudioClassification, TrainingArguments, Trainer, AutoFeatureExtractor

num_labels = len(id2label)
model = AutoModelForAudioClassification.from_pretrained(
    "facebook/wav2vec2-base", num_labels=num_labels, label2id=label2id, id2label=id2label
)

pytorch_model.bin:   0%|          | 0.00/380M [00:00<?, ?B/s]

Some weights of Wav2Vec2ForSequenceClassification were not initialized from the model checkpoint at facebook/wav2vec2-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'projector.bias', 'projector.weight', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
training_args = TrainingArguments(
    output_dir="Cafe_model",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=32,
    gradient_accumulation_steps=4,
    per_device_eval_batch_size=32,
    num_train_epochs=10,
    warmup_ratio=0.1,
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    # push_to_hub=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    tokenizer=processor.feature_extractor,
    compute_metrics=compute_metrics,
)

trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss,Accuracy
0,No log,1.269237,0.404762
2,1.157400,1.137223,0.559524
4,1.157400,1.050499,0.619048
6,0.985400,1.010320,0.583333
8,0.868000,1.006469,0.619048


TrainOutput(global_step=30, training_loss=1.0035688400268554, metrics={'train_runtime': 240.2345, 'train_samples_per_second': 17.483, 'train_steps_per_second': 0.125, 'total_flos': 1.916717986393703e+17, 'train_loss': 1.0035688400268554, 'epoch': 8.571428571428571})

In [ ]:
# # Define the model name for the Hugging Face Hub
# model_name = "Tasnuba2012848/Cafe_model"

# # Push the entire Trainer (including model, optimizer, scheduler, etc.) to the Hugging Face Hub
# trainer.push_to_hub(model_name)

# # Alternatively, if you specifically want to save and push the model and processor separately
# from transformers import Wav2Vec2Processor

# # Save both the model and processor locally
# model.save_pretrained("local_model_dir")
# processor.save_pretrained("local_model_dir")

# # Push the model and processor to the Hugging Face Hub (if you want separate pushes)
# model.push_to_hub(model_name)
# processor.push_to_hub(model_name)

events.out.tfevents.1730727098.4e8890c9db33.4363.0:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/378M [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

training_args.bin:   0%|          | 0.00/5.18k [00:00<?, ?B/s]

README.md:   0%|          | 0.00/1.99k [00:00<?, ?B/s]

README.md:   0%|          | 0.00/1.99k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Tasnuba2012848/Cafe_model/commit/e301f68b5b14ede9b1a1e5d21dc29a302fbb4c07', commit_message='Upload processor', commit_description='', oid='e301f68b5b14ede9b1a1e5d21dc29a302fbb4c07', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
from datasets import load_dataset
import torch
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

# Load model and processor from the Hugging Face Hub
model_name = "Tasnuba2012848/Cafe_model"
model = Wav2Vec2ForCTC.from_pretrained(model_name)
processor = Wav2Vec2Processor.from_pretrained(model_name)

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at Tasnuba2012848/Cafe_model and are newly initialized: ['lm_head.bias', 'lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
pip install soundfile

In [ ]:
print("Keys in test_dataset:", test_dataset.column_names)

Keys in test_dataset: ['id', 'length', 'emo', 'input_values', 'labels']


In [ ]:
# Import necessary libraries
import numpy as np
from sklearn.metrics import precision_recall_fscore_support
from transformers import Wav2Vec2ForSequenceClassification, Wav2Vec2Processor

# Load model and processor from the Hugging Face Hub
model_name = "Tasnuba2012848/Cafe_model"
model = Wav2Vec2ForSequenceClassification.from_pretrained(model_name)
processor = Wav2Vec2Processor.from_pretrained(model_name)

# Assuming the dataset is already created as before
test_dataset = dataset["test"]

# Run predictions on the test dataset
predictions = trainer.predict(test_dataset)
predicted_labels = np.argmax(predictions.predictions, axis=1)
true_labels = predictions.label_ids

# Function to calculate weighted and unweighted recall
def calculate_recall_metrics(true_labels, predicted_labels):
    # Compute per-class precision, recall, F1
    _, recall_per_class, _, _ = precision_recall_fscore_support(
        true_labels, predicted_labels, average=None, zero_division=0
    )

    # Compute weighted and unweighted (macro) recall
    weighted_recall = precision_recall_fscore_support(
        true_labels, predicted_labels, average='weighted', zero_division=0
    )[1]  # Index 1 is recall

    unweighted_recall = recall_per_class.mean()  # Mean recall for unweighted (macro)

    return {
        "weighted_recall": weighted_recall,
        "unweighted_recall": unweighted_recall
    }

# Display the recall metrics
recall_metrics = calculate_recall_metrics(true_labels, predicted_labels)
print("Weighted Recall:", recall_metrics["weighted_recall"])
print("Unweighted Recall:", recall_metrics["unweighted_recall"])

Weighted Recall: 0.6190476190476191
Unweighted Recall: 0.5416666666666666


In [ ]:
import logging
from transformers import TrainerCallback

# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

class PrintMetricsCallback(TrainerCallback):
    def on_epoch_end(self, args, state, control, metrics=None, **kwargs):
        if metrics is not None:
            # Extract training and evaluation metrics
            train_metrics = {k: v for k, v in metrics.items() if "train" in k}
            eval_metrics = {k: v for k, v in metrics.items() if "eval" in k}

            # Log the training and evaluation metrics
            logger.info(f"Training metrics after epoch {state.epoch}: {train_metrics}")
            logger.info(f"Evaluation metrics after epoch {state.epoch}: {eval_metrics}")

# Instantiate the Trainer with the custom callback
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    tokenizer=processor.feature_extractor,
    compute_metrics=compute_metrics,
    callbacks=[PrintMetricsCallback()]  # Add your custom callback here
)

# Start training
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
0,No log,1.213218,0.511905
2,0.968700,1.052150,0.583333
4,0.968700,0.954152,0.619048
6,0.790800,0.940440,0.630952
8,0.649800,0.912081,0.654762


TrainOutput(global_step=30, training_loss=0.8030888080596924, metrics={'train_runtime': 207.7254, 'train_samples_per_second': 20.219, 'train_steps_per_second': 0.144, 'total_flos': 1.916717986393703e+17, 'train_loss': 0.8030888080596924, 'epoch': 8.571428571428571})